<a href="https://colab.research.google.com/github/MaseedIrfanAli/DataFlow/blob/main/TumblingProcessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip3 install apache_beam

!pip3 install google-cloud-pubsub
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
import os
from apache_beam import window
from apache_beam.transforms.combiners import Count
import time
from apache_beam.transforms.window import TimestampedValue # Import from the correct module


serviceAccount = '/content/hadoop-431116-4cf85cb9d3ff.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= serviceAccount

input_subscription = 'projects/hadoop-431116/subscriptions/ratings-sub'

options= PipelineOptions()
options.view_as(StandardOptions).streaming= True

p = beam.Pipeline(options=options)

rating_count = 'projects/hadoop-431116/topics/rating_count_window'

def format(element):
  (movie,rating)=element
  return  "{r} rating for movieId {mId} in 5 seconds".format(r= rating, mId=movie).encode('utf-8')

pubsub_pipeline = (
    p
    | 'Read from pubsub topic' >> beam.io.ReadFromPubSub(subscription= input_subscription)
    | 'Split the records by comma' >> beam.Map(lambda row: row.decode("utf-8").split(','))
    | 'with Custo TimeStamp' >> beam.Map(lambda row: TimestampedValue(row, time.time())) # Use TimeStampedValue directly
    | 'Form Key Value Pair' >> beam.Map(lambda row: (row[1],float(row[2])))
    | 'Window' >> beam.WindowInto(window.FixedWindows(20))
    | 'Count the ratings' >> Count.PerKey()
    | 'Format' >> beam.Map(format)
    | 'Publish to output topic' >> beam.io.WriteToPubSub(rating_count)
)
result = p.run()
result.wait_until_finish()

KeyboardInterrupt: 